In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
import re 

df=pd.read_csv('phishing_site_urls.csv')
df=shuffle(df, random_state=200)

train=df.sample(frac=0.8, random_state=200) #random state is a seed value
test=df.drop(train.index)
n_data=train.shape[0]

tokens=set()
for i in range(len(train)):
    tokens.update(set(re.split(r'\.|/|\?|=',train['URL'].iloc[i].lower())))
    
vectorizer = CountVectorizer(vocabulary=tokens,min_df=1./n_data,max_df=1.0)

Ajustamos el vectorizador con una porcion de los datos de entrenamiento.

In [3]:
vectorizer.fit(train[:50000]['URL'])

CountVectorizer(min_df=2.2754319338668463e-06,
                vocabulary={'', '\x02$zn¿\x88', '\x05\t¯7\x8alø',
                            '\x08\x9d)&¡\x1fe¸\x8b\x1c'
                            '¢\x14à6r\x18d\xadnvy¨\x8bð«ñ3â¸%qñ+û\x93\x10è\x85'
                            '¸\x03\x12$¶gz{þ',
                            '\nø\x88\n\x85áö\x1d¯º\x9brê7¶\x15§',
                            '\x0e\x82âói<ý\x0b\x01ú1\x19ìþqdå¯5\x89ípç06',
                            '\x10\x0e0,°n\x10\x0e1,q!¹(7\x1c'
                            '66²²$¦\x9c¸2\x91\x01t¬!y((\x93\x8d\x87¦ùµ\x8...
                            '\x1c'
                            '\x88\x97\x89ôñ¿\x8cj²ã\x87crè\x147½\x0eýg\t'
                            '>¸4p!ÿ\x8fz\x95ôh°ú<af+¹×\x86\x04j÷|×¹æ\x87ô¾£\x95r\x85'
                            '\x0b'
                            '\x8f',
                            '\x1dü\x8dçàì\x81ëuvmq<º',
                            '\x1f\x9eók¦¾þ\x13)j&´^áyêïfg>|½}¸!\x11\x85'
                            '\

Ahora ajustamos el modelo mediante aprendizaje incremental, lo cual nos permite escalar el cómputo cuando no es posible almacenar los datos de entrenamiento en memoria. 

https://scikit-learn.org/stable/computing/scaling_strategies.html?highlight=out+core

* Pruebas con Bernoulli NB

In [ ]:
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB(alpha=0)

batch_size=500
n_batches=train.shape[0]//batch_size
for i in range(n_batches + 1):
    mini_batch = train[i*batch_size:(i+1)*batch_size]
    X_train = vectorizer.transform(mini_batch['URL'])
    y_train = (mini_batch['Label']=='bad').astype('int')
    if X_train.shape[0]>0:
        clf.partial_fit(X_train, y_train,classes=[0,1])


Una vez que entrenamos el modelo, podemos evaluar en datos de test y comparar con la etiqueta verdadera.


In [49]:
batch_size=500
n_batches=test.shape[0]//batch_size
y_hat=list()
y_true=list()
for i in range(n_batches + 1):
    mini_batch = test[i*batch_size:(i+1)*batch_size]
    X_test=vectorizer.transform(mini_batch['URL'])
    y_test=(mini_batch['Label']=='bad').astype('int')
    if X_test.shape[0]>0:
        y_pred=clf.predict(X_test)
        y_true.extend(y_test)
        y_hat.extend(y_pred)

In [51]:
target_names=train['Label'].unique()

In [52]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


print(classification_report(y_true, y_hat, target_names=target_names))

              precision    recall  f1-score   support

        good       0.98      0.99      0.98     78462
         bad       0.97      0.95      0.96     31407

    accuracy                           0.98    109869
   macro avg       0.97      0.97      0.97    109869
weighted avg       0.98      0.98      0.98    109869

